# Trend Centric Recommendation Model 

Importing necessary Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
dtype_spec = {
    'department_id': 'str',
    'Brand': 'str',
    'Description': 'str',
    'Image_URL': 'str',
    'Category_by_gender': 'str'
}
df = pd.read_csv('projectdataset.csv', dtype=dtype_spec, low_memory=False)
# Using only the first 31,583 rows
df = df.head(31583)



In [3]:
df.head(10)

,order_id,user_id,order_number,product_id,add_to_cart_order,reordered,department_id,Brand,Description,Image_URL,Category_by_gender,Category,action
0,2425083,49125,1,17,1,0,13,performax,Typographic Print Track Pants with Insert Pockets,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
1,2425083,49125,1,91,2,0,16,performax,Low-Top Lace-Up Sports Shoes,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
2,2425083,49125,1,36,3,0,16,teamspirit,Colorblock Crew-Neck T-shirt,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
3,2425083,49125,1,83,4,0,4,performax,Camouflage Print Joggers with Insert Pockets,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
4,2425083,49125,1,83,5,0,4,teamspirit,Reflective Colourblock Crew-Neck T-shirt,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
5,2425083,49125,1,91,6,0,16,teamspirit,Core Rib Joggers with Slip Pockets,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
6,2425083,49125,1,120,7,0,16,teamspirit,Core Varsity Polo T-shirt with Contrast Tipping,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
7,2425083,49125,1,59,8,0,15,performax,Typographic Print Running Crew-Neck T-shirt,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
8,2425083,49125,1,35,9,0,12,performax,Panelled Crew-Neck T-shirt,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view
9,1944304,162867,1,37,1,0,1,teamspirit,Core rib bottom track charcoal xs,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,view


In [4]:
action_map = {
    'view': 1,
    'add_to_cart': 2,
    'purchase': 3
}
df['action'] = df['action'].map(action_map)


In [24]:
df.head()

,order_id,user_id,order_number,product_id,add_to_cart_order,reordered,department_id,Brand,Description,Image_URL,Category_by_gender,Category,action
0,2425083,49125,1,17,1,0,13,performax,Typographic Print Track Pants with Insert Pockets,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,1.0
1,2425083,49125,1,91,2,0,16,performax,Low-Top Lace-Up Sports Shoes,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,1.0
2,2425083,49125,1,36,3,0,16,teamspirit,Colorblock Crew-Neck T-shirt,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,1.0
3,2425083,49125,1,83,4,0,4,performax,Camouflage Print Joggers with Insert Pockets,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,1.0
4,2425083,49125,1,83,5,0,4,teamspirit,Reflective Colourblock Crew-Neck T-shirt,https://assets.ajio.com/medias/sys_master/root...,Men,Activewear,1.0


In [7]:
# Verifying the dataset contains necessary columns
required_columns = ['user_id', 'product_id', 'action', 'Category_by_gender', 'Brand', 'Image_URL']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Dataset is missing required columns: {missing_columns}")



array([ 49125, 162867, 147243, ..., 144004, 101369,  70854], dtype=int64)

In [ ]:
# Initializing the LabelEncoder
label_encoders = {
    'user_id': LabelEncoder(),
    'product_id': LabelEncoder()
}

In [ ]:
# Fit and transform the user_id and product_id
df['user_id'] = label_encoders['user_id'].fit_transform(df['user_id'])
df['product_id'] = label_encoders['product_id'].fit_transform(df['product_id'])


In [ ]:
# Store original user IDs for validation
original_user_ids = label_encoders['user_id'].inverse_transform(df['user_id'].unique())
original_user_ids

In [8]:
# Aggregating actions for each user_id and product_id pair
df = df.groupby(['user_id', 'product_id', 'Category_by_gender', 'Brand', 'Image_URL']).agg({'action': 'sum'}).reset_index()

In [9]:
# Spliting the data into train and test sets
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
# Checking if the original user_id is present in the dataset
print(f"Checking if User ID 49125 is in the original dataset: {'Yes' if 49125 in df['user_id'].unique() else 'No'}")

Checking if User ID 49125 is in the original dataset: No


In [11]:
# Creating user-item interaction matrix
interaction_matrix = train.pivot_table(index='user_id', columns='product_id', values='action', aggfunc='sum').fillna(0)

In [12]:
# Computing the cosine similarity between users
user_similarity = cosine_similarity(interaction_matrix)

In [13]:
# Predicting function with image URLs
def recommend_products(user_id, num_recommendations=5):
    # Check if the user_id exists in the original dataset
    if user_id not in original_user_ids:
        raise ValueError(f"User ID {user_id} not found in the dataset.")
    # Encode the user_id
    encoded_user_id = label_encoders['user_id'].transform([user_id])[0]
    
    # Check if the encoded user_id exists in the interaction matrix
    if encoded_user_id not in interaction_matrix.index:
        raise ValueError(f"Encoded User ID {encoded_user_id} not found in the interaction matrix.")
    
    similarity_scores = user_similarity[encoded_user_id]
    similar_users = np.argsort(-similarity_scores)[1:]  # Exclude the user itself

    # Aggregate products added to cart or purchased by similar users
    product_scores = np.zeros(interaction_matrix.shape[1])
    for similar_user in similar_users:
        product_scores += interaction_matrix.iloc[similar_user].values * similarity_scores[similar_user]

    # Exclude products the user has already interacted with
    user_interactions = interaction_matrix.iloc[encoded_user_id].values
    product_scores[user_interactions > 0] = 0

    # Getting top recommendations
    recommended_product_ids = np.argsort(-product_scores)[:num_recommendations]
    recommended_product_ids = label_encoders['product_id'].inverse_transform(recommended_product_ids)
    
    # Getting the category, brand, and image URL of recommended products
    recommended_products = df[df['product_id'].isin(recommended_product_ids)]
    return recommended_products[['product_id', 'Category_by_gender', 'Brand', 'Image_URL']].drop_duplicates()

In [14]:
user_id = 147243  # This is the original user ID
try:
    recommendations = recommend_products(user_id, num_recommendations=5)
    print("Recommendations:")
    print(recommendations)
except ValueError as e:
    print(e)

Recommendations:
       product_id Category_by_gender               Brand  \
11             83                Men           performax   
53             83                Men            schumann   
57            120                Men         allen-solly   
70             83              Women             aurelia   
71             83              Women  avaasa-mix-n-match   
...           ...                ...                 ...   
31484          83                Men             netplay   
31497          83                Men           performax   
31511          83                Men                puma   
31542          83              Women  avaasa-mix-n-match   
31548          83              Women           performax   

                                               Image_URL  
11     https://assets.ajio.com/medias/sys_master/root...  
53     https://assets.ajio.com/medias/sys_master/root...  
57     https://assets.ajio.com/medias/sys_master/root...  
70     https://assets.ajio

In [15]:
from IPython.display import display, HTML

def display_recommendations(recommendations, num_items=3):
    html = '<h2>Recommendations</h2>'
    for _, row in recommendations.head(num_items).iterrows():
        html += f"""
        <div style="margin-bottom: 20px;">
            <img src="{row['Image_URL']}" alt="Product Image" style="width:100px;height:100px;">
            <p><strong>Product ID:</strong> {row['product_id']}</p>
            <p><strong>Category by Gender:</strong> {row['Category_by_gender']}</p>
            <p><strong>Brand:</strong> {row['Brand']}</p>
        </div>
        """
    display(HTML(html))


display_recommendations(recommendations, num_items=3)


In [66]:
# Trend Centric Recommendation
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix



In [67]:
dtype_spec = {
    'department_id': 'str',
    'Brand': 'str',
    'Description': 'str',
    'Image_URL': 'str',
    'Category_by_gender': 'str'
}

df2 = pd.read_csv('projectdataset2.csv', dtype=dtype_spec, low_memory=False)

df2 = df2.head(31583)

In [68]:

action_map = {
    'view': 1,
    'add_to_cart': 2,
    'purchase': 3
}
df['action'] = df['action'].map(action_map)

In [69]:
def get_recommendations(user_id):
    
    user_data = fashion_data[fashion_data['user_id'] == user_id]
    if user_data.empty:
        return None
    =
    return user_data.sort_values(by='ratings', ascending=False)['productId'].head(5).tolist()

In [70]:

# Calculating trendiness based on recent purchases or ratings count
df2['trendiness'] = df2.groupby('product_id')['ratings'].transform('count')
top_n_products = 10  # Number of top trending products to recommend
top_trending_products = df2[['product_id', 'Image_URL', 'trendiness']].drop_duplicates().nlargest(top_n_products, 'trendiness')


In [71]:
# Function to display images from the Image_URL column
def display_trending_fashion_images(top_trending_products):
    html = '<h2>Trending Fashion</h2>'
    for _, row in top_trending_products.iterrows():
        html += f"""
        <div style="display:inline-block; margin: 10px;">
            <img src="{row['Image_URL']}" alt="Product Image" style="width:150px;height:150px;">
            <p><strong>Product ID:</strong> {row['product_id']}</p>
        </div>
        """
    display(HTML(html))

In [72]:

display_trending_fashion_images(top_trending_products)